In [ ]:
#Make Necessary Import
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn import over_sampling
import glob

sns.set(rc={'figure.figsize':(16,8)})
sns.set_style("whitegrid")
sns.color_palette("dark")
plt.style.use("fivethirtyeight")

In [ ]:
files = [i for i in glob.glob('/Users/<USERID>/Desktop/Data/2020-2023/*.{}'.format('csv'))]
files.remove('/Users/<USERID>/Desktop/Data/2020-2023/JP_youtube_trending_data.csv')
files.remove('/Users/<USERID>/Desktop/Data/2020-2023/KR_youtube_trending_data.csv')
files.remove('/Users/<USERID>/Desktop/Data/2020-2023/MX_youtube_trending_data.csv')
files.remove('/Users/<USERID>/Desktop/Data/2020-2023/RU_youtube_trending_data.csv')
sorted(files)

others = ['/Users/<USERID>/Desktop/Data/2020-2023/JP_youtube_trending_data.csv', 
          '/Users/<USERID>/Desktop/Data/2020-2023/KR_youtube_trending_data.csv',
          '/Users/<USERID>/Desktop/Data/2020-2023/MX_youtube_trending_data.csv',
          '/Users/<USERID>/Desktop/Data/2020-2023/RU_youtube_trending_data.csv']

dfs = list()
for csv in files:
    df = pd.read_csv(csv, index_col='video_id')
    df['country'] = csv[21:23]
    dfs.append(df)

for csv in others:
    df = pd.read_csv(csv, index_col='video_id', encoding="ISO-8859-1")
    df['country'] = csv[21:23]
    dfs.append(df)

my_df = pd.concat(dfs)

my_df['trending_date']=pd.to_datetime(my_df['trending_date'],format='%Y-%m-%dT%H:%M:%SZ')
my_df['publishedAt']=pd.to_datetime(my_df['publishedAt'],format='%Y-%m-%dT%H:%M:%SZ')

my_df = my_df[my_df['trending_date'].notnull()]
my_df = my_df[my_df['publishedAt'].notnull()]

my_df = my_df.dropna(how='any',inplace=False, axis = 0)

my_df.insert(4, 'publish_date', my_df['publishedAt'].dt.date)
my_df['publishedAt'] = my_df['publishedAt'].dt.time
my_df=my_df.rename(columns = {'publishedAt':'publish_time'})
my_df=my_df.rename(columns = {'channelTitle':'channel_title'})

my_df_full = my_df.reset_index().sort_values('trending_date').set_index('video_id')
my_df = my_df.reset_index().sort_values('trending_date').drop_duplicates('video_id',keep='last').set_index('video_id')

my_df['desc_len']  = my_df['description'].str.len()
my_df['len_title']  = my_df['title'].str.len()

my_df['tags_list'] = my_df['tags'].str.split('|')
my_df['No_tags'] =my_df['tags_list'].apply(len)
my_df = my_df.drop(columns = 'tags_list')
my_df=my_df.rename(columns = {'categoryId':'category_id'})
data=my_df.rename(columns = {'view_count':'views'})